In [1]:
import urllib
import json
import sys
import codecs
import urllib.request
import pprint
import datetime
import time
from itertools import zip_longest

In [2]:
tag_name = "ALISグルメ企画"
new_tag_name= urllib.parse.quote(tag_name)
api_tag = 'https://alis.to/api/search/articles?tag='+new_tag_name
print(api_tag)

https://alis.to/api/search/articles?tag=ALIS%E3%82%B0%E3%83%AB%E3%83%A1%E4%BC%81%E7%94%BB


In [3]:
url_tag = urllib.request.urlopen(api_tag)

In [4]:
article_tags = json.loads(url_tag.read().decode("utf-8"))

In [5]:
article_ids = [article_tag.get('article_id') for article_tag in article_tags]
#article idを取り出す

In [6]:
user_ids = [article_tag.get('user_id') for article_tag in article_tags]
#user id を取り出す

In [7]:
api_article_ids = ["https://alis.to/api/articles/"+article_id for article_id in article_ids]
#article_idをAPIで取り出しやすい形にする

In [8]:
user_id_info_apis = ["https://alis.to/api/users/"+user_id+"/info" for user_id in user_ids]
#user_id_infoをAPIで取り出しやすい形にする

In [9]:
user_id_infos = [json.loads(urllib.request.urlopen(user_id_info_api).read().decode("utf-8")) for user_id_info_api in user_id_info_apis]
#/users/{user_id}/infoを取り出す

In [10]:
user_display_names = [user_id_info.get('user_display_name') for user_id_info in user_id_infos]
#user_display_name　を取り出す

In [11]:
likes_apis = ["https://alis.to/api/articles/"+article_id+"/likes" for article_id in article_ids]
#likes_apiをAPIで取り出しやすい形にする

In [12]:
likes = [json.loads(urllib.request.urlopen(likes_api).read().decode("utf-8")) for likes_api in likes_apis]

In [13]:
likes_counts = [like.get('count') for like in likes]
#Like数　を取り出す

In [14]:
articles_bodys = [json.loads(urllib.request.urlopen(api_article_id).read().decode("utf-8")) for api_article_id in api_article_ids]

In [15]:
def get_block(text, start_text, end_text):
    if not text.find(start_text) >= 0:
        return []
    new_texts = []
    for split_text in text.split(start_text):
        if split_text.find(end_text) >= 0:
            new_texts.append(split_text.split(end_text)[0])
    return new_texts

In [16]:
def get_block_tag(texts, start_text, end_text=None, tags=None):
    
    if not end_text:
        end_text = "<end>"
        
    if not tags:
        tags = ["企画タグ", "料理ジャンル", "店名", "住所"]
    
    texts = [text+end_text for text in texts]
    for tag in tags:
        texts = [text.replace(tag,end_text+tag) for text in texts]
    
    text = "".join(texts)
    
    if not text.find(start_text) >= 0:
        return []
    
    new_texts = []
    for split_text in text.split(start_text):
        if split_text.find(end_text) >= 0:
            new_texts.append(split_text.split(end_text)[0])
    
    remove_words = [":", "：", "<br>"]
    for remove_word in remove_words:
        new_texts = [new_text.replace(remove_word, "") for new_text in new_texts]
        
    new_texts = [new_text for new_text in new_texts if new_text]
    
    return new_texts

In [17]:
for articles_body in articles_bodys:
    texts = get_block(articles_body.get("body"), "<blockquote>", "</blockquote>")
    texts = [text for text in texts if text.find("〒") >= 0]
    project_names = get_block_tag(texts, "企画タグ")
    food_genres = get_block_tag(texts, "料理ジャンル")
    store_names = get_block_tag(texts, "店名")
    locations = get_block_tag(texts, "住所")
    for project_name, food_genre, store_name, location in zip_longest(project_names, food_genres, store_names, locations):
        tags_name=[project_name, food_genre, store_name, location]
        print(tags_name) 

['#秋の味覚絶品ピン', '#ラーメン\u3000#さんま', '拉麺 阿吽', '(〒含む)〒380-0928 長野県長野市若里４丁目１５−１８']
['#秋の味覚絶品ピン評価4', '#ラーメン\u3000#さんま', None, '(〒含む)〒380-0928 長野県長野市若里４丁目１５−１８']
['#秋の', '#ラーメン\u3000', None, '(〒含む)〒380-0928 若里４丁目１５−１８']


In [18]:
for (article_id, user_id) in zip(article_ids, user_ids):
    article_url="https://alis.to/"+str(user_id) +"/articles/"+ str(article_id)
    print(article_url)

https://alis.to/wand/articles/3Y1odX9j51qy
https://alis.to/wand/articles/3ldMkpzN4J87
https://alis.to/wand/articles/3LqV1qnyX1Dp


In [19]:
sheet_row_1 = ["企画名","料理ジャンル","店名","住所","記事URL","ユーザー名","いいね数"]
for i, sheet_rows_1 in enumerate(sheet_row_1):
   print(1, i+1, sheet_row_1[i])

1 1 企画名
1 2 料理ジャンル
1 3 店名
1 4 住所
1 5 記事URL
1 6 ユーザー名
1 7 いいね数
